# Demo of AzureNormalizer
Last edited: April 26, 2021

## Description
This function was build in support of the BIGIDEAS Lab's Covidentify Project (Duke University). The intention of the function is to extract, transform, and load the unstructured json data from the database that houses the Fitbit and Garmin API pulls into Microsoft Azure's ML Studio. 

## Background
   Because of the nature of Duke's Internal Review Board (IRB), the lab does not have direct control over the API pulls and thus using noSQL was not an option at this time. Without transformation, the json data was unusable to perform any modeling. The entire project was migrated to Azure during the month of Dec 2021, and this package was written to help transform and load the data for use in a Covid detection model.

## How to import the package

In [2]:
from AzureNormalizer import *

## Adding your personal credentials

In [3]:
import configparser
from azureml.core.authentication import ServicePrincipalAuthentication

#read ini file
config = configparser.ConfigParser()
config_filepath = 'workspace_config/cfg.ini'
config.read(config_filepath)

# svc_cfg = config['ServicePrincipalAuthentication']
ws_cfg = config['Azure_Workspace']

# svc_pr = ServicePrincipalAuthentication(
#     tenant_id = svc_cfg['tenant_id'],
#     service_principal_id = svc_cfg['service_principal_id'],
#     service_principal_password = svc_cfg['service_principal_password']
# )

## Load the data from a Azure Dataset

In [4]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

workspace = Workspace(
        subscription_id = ws_cfg['subscription_id'],
        resource_group = ws_cfg['resource_group'],
        workspace_name = ws_cfg['workspace_name'],
#         auth= svc_pr
    )
dataset = Dataset.get_by_name(workspace, name='covid_participant_positive')

### OPTIONAL: the function can be used with an Azure Dataset or with a pandas DataFrame (pandas is recommended) ###
df = dataset.to_pandas_dataframe()

Within the <code>AzureNormalizer</code> package there are two functions: 
* the main <code>json_normalizer</code> function: performs the transformation and loading work
* <code>whichPromptNames</code> function: provides a list of possible prompt names for the user to look at and iterate over
 
A demo of the functionality is shown below.

## Demo of <code>whichPromptNames</code>

From <code>whichPromptNames</code>'s documentation:
>This function is to help users identify which prompt names are available in json_normalizer function. This is manually inputted and if new prompt names are added, this needs to be updated.
>
>**INPUTS**<br>
><code>device</code>: either "garmin" or "fitbit".<br>
><code>timePeriod</code>: Either "historical" or "current". When users enrolled into study, their "historical" data was shared, "current" data is collected and uploaded to the prompt database.
>
>**OUTPUTS**<br>
>Outputs a list of prompt names.

In [5]:
# prompt = whichPromptNames("garmin", "historical") <- not incorporated yet
# print(">>> garmin, historical:", prompt)
prompt = whichPromptNames("garmin", "current")
print("\n>>> garmin, current:", prompt)
prompt = whichPromptNames("fitbit", "historical")
print("\n>>> fitbit, historical:", prompt)
prompt = whichPromptNames("fitbit", "current")
print("\n>>> fitbit, current:", prompt)


>>> garmin, current: ['covid_garmin_activity', 'covid_garmin_daily', 'covid_garmin_epoch', 'covid_garmin_sleep']

>>> fitbit, historical: ['covid_fitbit_heart_rate_intraday_backfill', 'covid_fitbit_steps_intraday_backfill', 'covid_fitbit_floors_intraday_backfill', 'covid_fitbit_floors_intraday_backfill', 'covid_fitbit_elevation_intraday_backfill', 'covid_fitbit_distance_intraday_backfill', 'covid_fitbit_calories_intraday_backfill', 'covid_fitbit_sleep_intraday_backfill']

>>> fitbit, current: ['heart_rate_intraday', 'sleep', 'calories_intraday', 'distance_intraday', 'elevation_intraday', 'floors_intraday', 'steps_intraday', 'r15_daily_sleep', 'weekly_r15_sleep_api', 'weekly_r15_water_api', 'weekly_r15_food_api']


> Using these lists, a user can use a for-loop to iterate over the possible prompt names to quickly generate the needed output datasets.

## Demo of <code>json_normalizer</code>

This code is built off of **Pandas'** <code>json_normalize</code>, but because of the nature of our nested jsons and exisitng relational data, more transformation work had to be incorporated before normalization could occur. Additonally, because each promptName has a different json structure, different keyword arguments needed to be implemented for each promptName to make a process that was generalizable for the entire data.

From <code>json_normalizer</code>'s documentation:
>This function will extract an Azure Dataset from a Azure Dataset Consume\
>transform the nested json into a relational table and load the transformed\
>table directly into the Azure dataSets Assets list.
>
>**INPUTS**<br>
><code>device</code>: either "garmin" or "fitbit"<br>
><code>timePeriod</code>: Either "historical" or "current". When users enrolled into study, their "historical" data was shared, "current" data is collected and uploaded to the prompt database.<br>
><code>promptName</code>: name of the type of data collected.<br>
><code>data</code>: dataset from the targeted Azure DataSet Consume.<br>
><code>aggSummary</code> (optional): for some garmin promptNames, it makes more sense to display the data in the aggregated summaries form, which is one level above the default summaries.<br>
><code>ignoreInputs</code> (optional): if turned on, skips the default user inputs.<br>
><code>prefix</code> (optional): used in conjunction when 'ignoreInputs' is turned on to set the uploaded filename's prefix.<br>
><code>pandas</code> (optional): Allows using a pandas dataFrame instead of using a Azure dataset. There is a performance boost when converting to a pandas dataFrame outside of the function.<br>
>
>**OUTPUTS**<br>
>Automatically uploads the transformed data into the Azure dataSets Assets list.

### json_normalizer, Demo 1: Running the function from a Pandas dataframe

In [7]:
json_normalizer("fitbit", "current", prompt[1], data=df, pandas=True, pipeline=False, workspace=workspace)

>>> The shape of the output dataframe is:  (135, 7)
>>> The number of rows that did not have data are: 0


Do you want see a sample? [y/n] y


    sleep captured_at  summary.totalMinutesAsleep  summary.totalSleepRecords  \
9      []  2020-07-24                           0                          0   
83     []  2020-11-07                           0                          0   
97     []  2020-11-04                           0                          0   
79     []  2020-09-28                           0                          0   
29     []  2020-08-12                           0                          0   
..    ...         ...                         ...                        ...   
102    []  2020-10-22                           0                          0   
96     []  2020-10-17                           0                          0   
134    []  2020-11-28                           0                          0   
73     []  2020-09-23                           0                          0   
110    []  2020-10-30                           0                          0   

     summary.totalTimeInBed  uniqueRowI

Do you want to upload? [y/n] y
What do you want the prefix for the file to be? demo


>>> Starting upload...
./output_csv/demo-sleep.csv
Uploading an estimated of 1 files
Uploading ./output_csv/demo-sleep.csv
Uploaded ./output_csv/demo-sleep.csv, 1 files out of an estimated total of 1
Uploaded 1 files
>>> Upload Successful!


>**Notice that there are user input options above. These user inputs are:**<br>
>   1. Seeing a random sample of 10 lines of the data before uploading. If there are less than 10 lines, the data will be randomly repeated.
>   2. An option to upload the data.
>   3. If uploading, a prefix can be designated to help distinguish the source of the data.

> **Below is a screenshot of Azure's ML Studio to show where the dataset ends up:**<br>

<center><img src="./assets/dataset_in_MLPortal.png" alt="Dataset location in portal" style="width: 1000px;"/></center>

> **This screenshot shows that the code auto-generates the description with the input info:**<br>

<center><img src="./assets/dataset_description.png" alt="Dataset description" style="width: 1000px;"/></center>

### json_normalizer, Demo 2: What happens if you try to use the wrong inputs

In [9]:
json_normalizer("fitbit", "historical", prompt[1], data=df, pandas=True, workspace=workspace)

ValueError: No data in output dataframe. Check inputs.

The custom exception was added for when a the data outputs a dataframe shape of (0,0). If the the entire dataframe has no data, most likely the inputs are incorrect. In the case above, prompt[1] was from the <code>fitbit</code> <code>current</code> list, so the promptname  <code>'covid_fitbit_steps_intraday_backfill'</code> could not be found in the appropriate keyword-argument list to run <code>json_normalize</code>.

However, there are times when the data has missing rows for specific datasets and can handle a some missing data. See below:

In [10]:
json_normalizer("fitbit", "historical", "covid_fitbit_heart_rate_intraday_backfill", data=df, pandas=True, workspace=workspace)

>>> The shape of the output dataframe is:  (217540, 9)
>>> The number of rows that did not have data are: 11


Do you want see a sample? [y/n] y


        hRZ-max  hRZ-min      hRZ-name  hRZ-minutes  hRZ-caloriesOut  \
5529        115       99      Fat Burn          5.0         22.69170   
160794      126      104        Cardio         10.0         35.26236   
95097       104       74      Fat Burn        941.0        884.27764   
72546       125      103        Cardio          1.0          3.66252   
68615       220      125          Peak          0.0          0.00000   
...         ...      ...           ...          ...              ...   
11037       103       74      Fat Burn        916.0        897.47664   
54330       126      104        Cardio          0.0          0.00000   
153064       74       30  Out of Range        411.0        334.16514   
166268       98       30  Out of Range       1440.0       1146.52800   
29646       125      103        Cardio          1.0          3.18480   

       captured_at.dateTime activities-heart.value.restingHeartRate  \
5529             2020-10-23                                     

Do you want to upload? [y/n] n


### json_normalizer, Demo 3: What is aggSummary?

For some promptNames, especially the Garmin promptNames, there is a aggregated summary that encasulates the entire observation period, and a nested summary that captures more granular observation periods. To illustrate this, the json structure of the promptName <code>covid_garmin_activity</code> for the 'current' <code>timePeriod</code> is shown below. <code>Summary</code> is highlighted in green and <code>aggregatedSummary</code> is highlighted in purple.

<center><img src="./assets/summaryAggSummary_compare.png" alt="Comparision of Summary and AggSummary" style="width: 600px;"/></center>

In this dataset, an output for a single line would look like this:

#### <code>aggregatedSummary</code> of covid_garmin_activity:

In [11]:
import pandas as pd
import numpy as np
from pandas import json_normalize
import json

promptValue = "covid_garmin_activity"
chars = '"'
subsetForTransformation = df[(df.prompt_name == promptValue)]

pd_current_json_value = pd.DataFrame(json_normalize(data=json.loads(subsetForTransformation["json_value"].iloc[93].replace("\\","").strip(chars)),))

pd_current_json_value

,activities,captured_at,aggregatedSummary.durationInSeconds,aggregatedSummary.averageHeartRateInBeatsPerMinute,aggregatedSummary.distanceInMeters,aggregatedSummary.steps
0,"[{'summaryId': '5133495146-detail', 'summary':...",2020-06-23,2402,135,0,0


#### <code>Summary</code> of covid_garmin_activity:

In [12]:
pd_current_json_value = pd.DataFrame(json_normalize(
                                        data=json.loads(subsetForTransformation["json_value"].iloc[93].replace("\\","").strip(chars)),
                                        record_path =  "activities", meta =  "captured_at",))
pd_current_json_value

,summaryId,samples,laps,summary.durationInSeconds,summary.startTimeInSeconds,summary.startTimeOffsetInSeconds,summary.activityType,summary.averageHeartRateInBeatsPerMinute,summary.activeKilocalories,summary.deviceName,summary.maxHeartRateInBeatsPerMinute,captured_at
0,5133495146-detail,"[{'startTimeInSeconds': 1592924840, 'elevation...",[{'startTimeInSeconds': 1592924840}],2402,1592924840,3600,INDOOR_CARDIO,135,387,vivoactive3,175,2020-06-23


The reason that this option exists is that for some promptNames, it makes more sense to use the aggregatedSummary compared to looking at the underlying data. An example of this is if you wanted the step count of an entire day.

### json_normalizer, Demo 4: Ignoring Inputs

If a user wants to iterate over a list prompt names, one may want to turn off the [y/n] user inputs. To do so, set <code>ignoreInputs</code> to <code>True</code>. Normally, one would still want to set the desired prefix of the filepath. Thus, input the desired prefix into <code>prefix</code>.

This is demonstrated below on all the promptNames in Garmin in the <code>current</code> time period (Note: promptName <code>covid_garmin_daily</code>  may take a while to run).

In [13]:
prompt = whichPromptNames("garmin", "current")
#print("\n>>> garmin, current:", prompt)
for pmt in prompt:
    print(f"\n>>>The prompt name is: {pmt}")
    json_normalizer("garmin", "current", pmt, data=df, pandas=True, ignoreInputs=True, prefix="demo_2", workspace=workspace)


>>>The prompt name is: covid_garmin_activity
>>> The shape of the output dataframe is:  (143, 28)
>>> The number of rows that did not have data are: 0
>>> Starting upload...
./output_csv/demo_2-covid_garmin_activity.csv
Uploading an estimated of 1 files
Uploading ./output_csv/demo_2-covid_garmin_activity.csv
Uploaded ./output_csv/demo_2-covid_garmin_activity.csv, 1 files out of an estimated total of 1
Uploaded 1 files
>>> Upload Successful!

>>>The prompt name is: covid_garmin_daily
>>> The shape of the output dataframe is:  (4661, 35)
>>> The number of rows that did not have data are: 0
>>> Starting upload...
./output_csv/demo_2-covid_garmin_daily.csv
Uploading an estimated of 1 files
Uploading ./output_csv/demo_2-covid_garmin_daily.csv
Uploaded ./output_csv/demo_2-covid_garmin_daily.csv, 1 files out of an estimated total of 1
Uploaded 1 files
>>> Upload Successful!

>>>The prompt name is: covid_garmin_epoch
>>> The shape of the output dataframe is:  (18431, 16)
>>> The number of row

<center><img src="./assets/listOfGarminDatasets.png" alt="Comparision of Summary and AggSummary" style="width: 600px;"/></center>

### json_normalizer, Demo 5: Azure Dataset versus Pandas DataFrame

Although <code>json_normalizer</code> can use Azure Datasets, it is advised to use a pandas dataframe instead because the time it takes to run using an Azure Dataset is much greater than a pandas dataframe. See below:

In [14]:
import time

start = time.time()

json_normalizer("fitbit", "historical", "covid_fitbit_steps_intraday_backfill", data=dataset,  ignoreInputs=True, prefix="demo_3", pandas=False, workspace=workspace)

end = time.time()

print(f"Time ellapsed: {end - start:.5f} seconds.")

>>> The shape of the output dataframe is:  (58400, 4)
>>> The number of rows that did not have data are: 0
>>> Starting upload...
./output_csv/demo_3-covid_fitbit_steps_intraday_backfill.csv
Uploading an estimated of 1 files
Uploading ./output_csv/demo_3-covid_fitbit_steps_intraday_backfill.csv
Uploaded ./output_csv/demo_3-covid_fitbit_steps_intraday_backfill.csv, 1 files out of an estimated total of 1
Uploaded 1 files
>>> Upload Successful!
Time ellapsed: 52.95737 seconds.


In [15]:
import time

start = time.time()

json_normalizer("fitbit", "historical", "covid_fitbit_steps_intraday_backfill", data=df,  ignoreInputs=True, prefix="demo_3", pandas=True, workspace=workspace)

end = time.time()

print(f"Time ellapsed: {end - start:.5f} seconds.")

>>> The shape of the output dataframe is:  (58400, 4)
>>> The number of rows that did not have data are: 0
>>> Starting upload...
>>> Upload Successful!
Time ellapsed: 2.69776 seconds.
